In [0]:
from keras.models import Model
from keras.layers import Input, Activation, merge, Dense, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, add, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.models import load_model
from keras.datasets import cifar10
from keras.utils import np_utils
import numpy as np


Using TensorFlow backend.


In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [0]:

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')


X_train /= 255.0
X_test /= 255.0


n_classes = 10
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)

In [0]:



def _shortcut(inputs, residual):
    n_filters = residual._keras_shape[3]
    shortcut = Convolution2D(n_filters, (1,1), strides=(1,1), padding='valid')(inputs)
  
    return add([shortcut, residual])



def _resblock(n_filters1, n_filters2, strides=(1,1)):
    def f(input):
        x = Convolution2D(n_filters1, (1,1), strides=strides,
                                      kernel_initializer='he_normal', padding='same')(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Convolution2D(n_filters1, (3,3), strides=strides,
                                      kernel_initializer='he_normal', padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Convolution2D(n_filters2, (1,1), strides=strides,
                                      kernel_initializer='he_normal', padding='same')(x)
        x = BatchNormalization()(x)
   
        return _shortcut(input, x)
    
    return f


def resnet():
    inputs = Input(shape=(32, 32, 3))
  
    x = Convolution2D(32, (7,7), strides=(1,1),
                    kernel_initializer='he_normal', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2,2), padding='same')(x)
  
 
  
    x = _resblock(n_filters1=8, n_filters2=32)(x)
    x = _resblock(n_filters1=12, n_filters2=48)(x)
    x = _resblock(n_filters1=16, n_filters2=64)(x)
    x = _resblock(n_filters1=20, n_filters2=80)(x)
    x = _resblock(n_filters1=24, n_filters2=96)(x)
    x = _resblock(n_filters1=28, n_filters2=112)(x)
    x = _resblock(n_filters1=32, n_filters2=128)(x)
    x = _resblock(n_filters1=36, n_filters2=144)(x)
    x = _resblock(n_filters1=40, n_filters2=160)(x)
    x = MaxPooling2D(strides=(2,2))(x) 
    x = _resblock(n_filters1=44, n_filters2=176)(x)
    x = _resblock(n_filters1=48, n_filters2=192)(x)
    x = _resblock(n_filters1=52, n_filters2=208)(x) 
    x = _resblock(n_filters1=56, n_filters2=224)(x)
    x = _resblock(n_filters1=60, n_filters2=240)(x)
    x = _resblock(n_filters1=64, n_filters2=256)(x) 

  
    x = GlobalAveragePooling2D()(x)
    x = Dense(10, kernel_initializer='he_normal', activation='softmax')(x)
  
    model = Model(inputs=inputs, outputs=x)
    return model

model = resnet()

adam = Adam()
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   4736        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [0]:
batch_size = 50
epochs = 60


h = model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2,)

Train on 40000 samples, validate on 10000 samples
Epoch 1/60
40000/40000 [==============================] - 122s 3ms/step - loss: 1.5614 - acc: 0.4512 - val_loss: 1.5788 - val_acc: 0.4619
Epoch 2/60
40000/40000 [==============================] - 110s 3ms/step - loss: 1.2226 - acc: 0.5624 - val_loss: 1.3885 - val_acc: 0.5488
Epoch 3/60
40000/40000 [==============================] - 110s 3ms/step - loss: 1.0699 - acc: 0.6214 - val_loss: 1.2827 - val_acc: 0.5484
Epoch 4/60
40000/40000 [==============================] - 109s 3ms/step - loss: 0.9608 - acc: 0.6606 - val_loss: 1.3223 - val_acc: 0.5601
Epoch 5/60
40000/40000 [==============================] - 110s 3ms/step - loss: 0.8816 - acc: 0.6931 - val_loss: 1.6805 - val_acc: 0.5463
Epoch 6/60
40000/40000 [==============================] - 110s 3ms/step - loss: 0.8281 - acc: 0.7103 - val_loss: 1.1579 - val_acc: 0.6088
Epoch 7/60
40000/40000 [==============================] - 110s 3ms/step - loss: 0.7755 - acc: 0.7303 - val_loss: 1.2183 - 